In [1]:
import pandas as pd
lang='Mal'
#lang='tamil'
#lang='kannada'

In [2]:
train=pd.read_csv('Saco2021DS/'+lang+'_sentiment_full_train.tsv',sep='\t')
dev=pd.read_csv('Saco2021DS/'+lang+'_sentiment_full_dev.tsv',sep='\t')
test=pd.read_csv('Saco2021DS/'+lang+'_sentiment_full_test_withoutlabels.tsv',sep='\t')

In [3]:
len(train),len(dev),len(test)

(15888, 1766, 1962)

In [4]:
train.category.value_counts()

Positive          6421
unknown_state     5279
Negative          2105
not-malayalam     1157
Mixed_feelings     926
Name: category, dtype: int64

In [5]:
dev.category.value_counts()

Positive          706
unknown_state     580
Negative          237
not-malayalam     141
Mixed_feelings    102
Name: category, dtype: int64

In [6]:
test.head()

,id,text
0,Mal_1,Teaserinu kurach samayamkoodi mathram. Cant wa...
1,Mal_2,അപ്പോൾ കഥയുടെ റൂട്ട് മാറിയല്ലോ...
2,Mal_3,മൂത്തോൻ ട്രൈലെർ Trending List വരാത്തത് എന്താ ?
3,Mal_4,Nowadays 944k views is considered as 1M views ...
4,Mal_5,Maass.trailer ennu paranja ithaanU makkale.......


In [7]:
emoji=open('emoji.txt',encoding='utf-8').read()
unwanted=['ـ','_','"','！',"'",'=','@','&','%','.',',',':','\\','$','^','<','>','!','?','{','}',';','\n','\t','(',')','[',']','/','*','+','#','\u200c','\ufeff','-','_','|']
unwanted1=["''","\u200d"]
emo=""

for i in emoji:
  if i not in unwanted and i not in unwanted1:
    emo=emo+i
emo=list(set(emo))

#!pip install indic-nlp-library
#!pip install stopwordsiso
#!pip install emoji
import emoji as em
import pandas as pd

import string 
import re
#from indicnlp.tokenize import indic_tokenize
#import stopwordsiso as stopwords

#import nltk
#nltk.download('stopwords')
#nltk.download('wordnet')
def clean_text(text):
    '''repeat_pattern = re.compile(r'(\w)\1*') 
    match_substitution = r'\1' 
    text=repeat_pattern.sub(match_substitution,text)'''
    #text=text.replace('\n',' ')
    stpwrds=[]#open('/content/drive/MyDrive/Constraint@AAAI2021/stopwords-hi.txt').read().split('\n')
    punctuations = '''!()-[]{};:'"\,<>./?#$=%^012|345678”€¥`9‘+@“’&*_~''' + "[\.]"*3+'\U000e0067'
    te=''
    for char in text:
        if char in emo:
            char=em.demojize(char)
            char=char.replace(":","").replace('_',' ')
            
            #char=char+   
        te=te+char
    text="".join([char for char in te if char not in punctuations and len(char)>0])   #string.punctuation
    #print(text)
    sent=[]
    text=text.split(' ')
    
    for word in text:

        if word not in stpwrds and len(word)>2:
            #word=ps.stem(word)
            #word=wn.lemmatize(word)
            if not (word.startswith('@')):
              sent.append(word.lower())
    return str(' '.join(sent).strip())

In [8]:

train['cleaned']=[clean_text(x) for x in train.text]
dev['cleaned']=[clean_text(x) for x in dev.text]
test['cleaned']=[clean_text(x) for x in test.text]



In [9]:
#!pip install xgboost

In [10]:
from sklearn import model_selection, preprocessing, metrics, svm
from sklearn.feature_extraction.text import CountVectorizer,TfidfVectorizer
from sklearn import decomposition, ensemble

import pandas as pd
import numpy as np

import string

#import matplotlib.pyplot as plt
#import seaborn as sns
#%matplotlib inline

import xgboost
#from keras.preprocessing import text, sequence
#from keras import layers, models, optimizers
from sklearn.feature_extraction.text import CountVectorizer,TfidfVectorizer,HashingVectorizer



In [11]:
sentences=list(train.cleaned)
from scipy.sparse import hstack
word_vectorizer = TfidfVectorizer(
    sublinear_tf=True,
    strip_accents='unicode',
    analyzer='word',
    ngram_range=(1,3))
word_vectorizer.fit(sentences)

char_vectorizer = TfidfVectorizer(
    sublinear_tf=True,
    strip_accents='unicode',
    min_df=4,
    analyzer='char',
    ngram_range=(1,5))
char_vectorizer.fit(sentences)





train_word_features = word_vectorizer.transform(train.cleaned)
dev_word_features = word_vectorizer.transform(dev.cleaned)
test_word_features = word_vectorizer.transform(test.cleaned)


train_char_features = char_vectorizer.transform(train.cleaned)
dev_char_features = char_vectorizer.transform(dev.cleaned)
test_char_features = char_vectorizer.transform(test.cleaned)



x_train = hstack([train_char_features, train_word_features])#,train_sn_features])
x_dev = hstack([dev_char_features, dev_word_features])#,test_sn_features])
x_test = hstack([test_char_features, test_word_features])#,test_sn_features])



y_train=train.category
y_dev=dev.category

**only char ngrams**

In [12]:
from sklearn.ensemble import VotingClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.svm import LinearSVC,SVC
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report, f1_score


lr=LogisticRegression(C=20)
lr.fit(train_char_features, y_train)
y_pred = lr.predict(dev_char_features) 
print(classification_report(y_dev, y_pred,digits=2))

                precision    recall  f1-score   support

Mixed_feelings       0.53      0.35      0.42       102
      Negative       0.74      0.60      0.66       237
      Positive       0.78      0.86      0.82       706
 not-malayalam       0.86      0.82      0.84       141
 unknown_state       0.73      0.74      0.74       580

      accuracy                           0.75      1766
     macro avg       0.73      0.68      0.70      1766
  weighted avg       0.75      0.75      0.75      1766



/home/vijay/Desktop/notebook/jupyterenv/lib/python3.8/site-packages/sklearn/linear_model/_logistic.py:763: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


In [13]:
test.head()

,id,text,cleaned
0,Mal_1,Teaserinu kurach samayamkoodi mathram. Cant wa...,teaserinu kurach samayamkoodi mathram cant wait
1,Mal_2,അപ്പോൾ കഥയുടെ റൂട്ട് മാറിയല്ലോ...,അപ്പോൾ കഥയുടെ റൂട്ട് മാറിയല്ലോ
2,Mal_3,മൂത്തോൻ ട്രൈലെർ Trending List വരാത്തത് എന്താ ?,മൂത്തോൻ ട്രൈലെർ trending list വരാത്തത് എന്താ
3,Mal_4,Nowadays 944k views is considered as 1M views ...,nowadays views considered views kerala
4,Mal_5,Maass.trailer ennu paranja ithaanU makkale.......,maasstrailer ennu paranja ithaanu makkalekatta...


##predictions 

In [14]:
y_pred = lr.predict(test_char_features)
test1=test
test1['label']=y_pred
test1=test1.drop(['cleaned'],axis=1)
test1.to_csv(lang+'_lr_char.tsv',sep='\t')
print(len(test1))
test1.head()


1962


,id,text,label
0,Mal_1,Teaserinu kurach samayamkoodi mathram. Cant wa...,unknown_state
1,Mal_2,അപ്പോൾ കഥയുടെ റൂട്ട് മാറിയല്ലോ...,unknown_state
2,Mal_3,മൂത്തോൻ ട്രൈലെർ Trending List വരാത്തത് എന്താ ?,Positive
3,Mal_4,Nowadays 944k views is considered as 1M views ...,not-malayalam
4,Mal_5,Maass.trailer ennu paranja ithaanU makkale.......,Positive


In [15]:
'''import pickle
#!rm -r models
!mkdir models

filename = 'models/charvec.sav'
pickle.dump(char_vectorizer, open(filename, 'wb'))

filename = 'models/lr.sav'
pickle.dump(lr, open(filename, 'wb'))'''

"import pickle\n#!rm -r models\n!mkdir models\n\nfilename = 'models/charvec.sav'\npickle.dump(char_vectorizer, open(filename, 'wb'))\n\nfilename = 'models/lr.sav'\npickle.dump(lr, open(filename, 'wb'))"

In [16]:

lsvc=SVC(kernel='linear',probability=True)
lsvc.fit(train_char_features, y_train)
y_pred = lsvc.predict(dev_char_features) 
print(classification_report(y_dev, y_pred,digits=2))

                precision    recall  f1-score   support

Mixed_feelings       0.59      0.29      0.39       102
      Negative       0.75      0.57      0.65       237
      Positive       0.76      0.84      0.79       706
 not-malayalam       0.85      0.84      0.84       141
 unknown_state       0.72      0.77      0.75       580

      accuracy                           0.75      1766
     macro avg       0.73      0.66      0.68      1766
  weighted avg       0.74      0.75      0.74      1766



In [17]:
y_pred = lsvc.predict(test_char_features)
test1=test
test1['label']=y_pred
test1=test1.drop(['cleaned'],axis=1)
test1.to_csv(lang+'_lsvc_char.tsv',sep='\t')
print(len(test1))
test1.head()


1962


,id,text,label
0,Mal_1,Teaserinu kurach samayamkoodi mathram. Cant wa...,unknown_state
1,Mal_2,അപ്പോൾ കഥയുടെ റൂട്ട് മാറിയല്ലോ...,unknown_state
2,Mal_3,മൂത്തോൻ ട്രൈലെർ Trending List വരാത്തത് എന്താ ?,Positive
3,Mal_4,Nowadays 944k views is considered as 1M views ...,not-malayalam
4,Mal_5,Maass.trailer ennu paranja ithaanU makkale.......,Positive


In [18]:
'''filename = 'models/lsvc.sav'
pickle.dump(lsvc, open(filename, 'wb'))'''

"filename = 'models/lsvc.sav'\npickle.dump(lsvc, open(filename, 'wb'))"

In [19]:
from sklearn.neural_network import MLPClassifier
mlp=MLPClassifier(hidden_layer_sizes=(150,100,50), max_iter=300,activation = 'relu',solver='adam',random_state=1)
mlp.fit(train_char_features, y_train)
y_pred = mlp.predict(dev_char_features) 
print(classification_report(y_dev, y_pred,digits=2))

                precision    recall  f1-score   support

Mixed_feelings       0.44      0.36      0.40       102
      Negative       0.61      0.57      0.59       237
      Positive       0.77      0.81      0.79       706
 not-malayalam       0.84      0.75      0.79       141
 unknown_state       0.70      0.72      0.71       580

      accuracy                           0.72      1766
     macro avg       0.67      0.64      0.65      1766
  weighted avg       0.71      0.72      0.71      1766



In [20]:
y_pred = mlp.predict(test_char_features)
test1=test
test1['label']=y_pred
test1=test1.drop(['cleaned'],axis=1)
test1.to_csv(lang+'_mlp_char.tsv',sep='\t')
print(len(test1))
test1.head()


1962


,id,text,label
0,Mal_1,Teaserinu kurach samayamkoodi mathram. Cant wa...,not-malayalam
1,Mal_2,അപ്പോൾ കഥയുടെ റൂട്ട് മാറിയല്ലോ...,unknown_state
2,Mal_3,മൂത്തോൻ ട്രൈലെർ Trending List വരാത്തത് എന്താ ?,Positive
3,Mal_4,Nowadays 944k views is considered as 1M views ...,not-malayalam
4,Mal_5,Maass.trailer ennu paranja ithaanU makkale.......,Positive


In [21]:
'''filename = 'models/mlp.sav'
pickle.dump(mlp, open(filename, 'wb'))'''

"filename = 'models/mlp.sav'\npickle.dump(mlp, open(filename, 'wb'))"

In [22]:
'''import xgboost as xgb

xgb_clf=xgb.XGBClassifier(max_depth=20, n_estimators=80, learning_rate=0.1, colsample_bytree=.7, gamma=.01, reg_alpha=4, objective='multi:softmax')

xgb_clf.fit(train_char_features, y_train)
y_pred =xgb_clf.predict(dev_char_features) 
print(classification_report(y_dev, y_pred,digits=2))'''

"import xgboost as xgb\n\nxgb_clf=xgb.XGBClassifier(max_depth=20, n_estimators=80, learning_rate=0.1, colsample_bytree=.7, gamma=.01, reg_alpha=4, objective='multi:softmax')\n\nxgb_clf.fit(train_char_features, y_train)\ny_pred =xgb_clf.predict(dev_char_features) \nprint(classification_report(y_dev, y_pred,digits=2))"

In [23]:
'''

filename = 'models/xgb.sav'
pickle.dump(xgb_clf, open(filename, 'wb'))'''

"\n\nfilename = 'models/xgb.sav'\npickle.dump(xgb_clf, open(filename, 'wb'))"

In [24]:
from sklearn.ensemble import VotingClassifier
from sklearn.neural_network import MLPClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.svm import LinearSVC,SVC
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report, f1_score
from sklearn.multiclass import OneVsRestClassifier
import xgboost as xgb
from sklearn.linear_model import SGDClassifier



#mnb=MultinomialNB()
mlp=MLPClassifier(hidden_layer_sizes=(150,100,50), max_iter=300,activation = 'relu',solver='adam',random_state=1)
xgb_clf=SVC(kernel='linear',probability=True)
lr=LogisticRegression()
estimators=[('lr', lr), ('mlp', mlp),('xgb', xgb_clf)]
clf = VotingClassifier(estimators, voting='soft')
ensemble =clf


ensemble.fit(train_char_features, y_train)



/home/vijay/Desktop/notebook/jupyterenv/lib/python3.8/site-packages/sklearn/linear_model/_logistic.py:763: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


VotingClassifier(estimators=[('lr', LogisticRegression()),
                             ('mlp',
                              MLPClassifier(hidden_layer_sizes=(150, 100, 50),
                                            max_iter=300, random_state=1)),
                             ('xgb', SVC(kernel='linear', probability=True))],
                 voting='soft')

In [25]:
y_pred = ensemble.predict(dev_char_features) 
print(classification_report(y_dev, y_pred,digits=4))

                precision    recall  f1-score   support

Mixed_feelings     0.5286    0.3627    0.4302       102
      Negative     0.6731    0.5907    0.6292       237
      Positive     0.7704    0.8272    0.7978       706
 not-malayalam     0.8421    0.7943    0.8175       141
 unknown_state     0.7253    0.7466    0.7358       580

      accuracy                         0.7395      1766
     macro avg     0.7079    0.6643    0.6821      1766
  weighted avg     0.7343    0.7395    0.7352      1766



In [26]:
y_pred = ensemble.predict(test_char_features)
test1=test
test1['label']=y_pred
test1=test1.drop(['cleaned'],axis=1)
test1.to_csv(lang+'_ensemble_char.tsv',sep='\t')
print(len(test1))
test1.head()


1962


,id,text,label
0,Mal_1,Teaserinu kurach samayamkoodi mathram. Cant wa...,unknown_state
1,Mal_2,അപ്പോൾ കഥയുടെ റൂട്ട് മാറിയല്ലോ...,unknown_state
2,Mal_3,മൂത്തോൻ ട്രൈലെർ Trending List വരാത്തത് എന്താ ?,Positive
3,Mal_4,Nowadays 944k views is considered as 1M views ...,not-malayalam
4,Mal_5,Maass.trailer ennu paranja ithaanU makkale.......,Positive


**Syllabel ngrams**

In [34]:
'''!git clone https://github.com/libindic/syllabalizer.git
import os
os.chdir('syllabalizer')
!python setup.py sdist
%cd ../
!pip install libindic.utils
!pip install './syllabalizer/dist/libindic-syllabifier-0.0.1.dev28.tar.gz'
'''

"!git clone https://github.com/libindic/syllabalizer.git\nimport os\nos.chdir('syllabalizer')\n!python setup.py sdist\n%cd ../\n!pip install libindic.utils\n!pip install './syllabalizer/dist/libindic-syllabifier-0.0.1.dev28.tar.gz'\n"

In [35]:
def generateNgram(words, n=1):
    gram = dict()
    
    # Some helpers to keep us crashing the PC for now
    #assert n > 0 and n < 100
    
    # Populate N-gram dictionary
    for i in range(len(words)-(n-1)):
        key = tuple(words[i:i+n])
        if gram in key:
            gram[key] += 1
        else:
            gram[key] = 1

    # Turn into a list of (word, count) sorted by count from most to least
    #gram = sorted(gram.items(), key=lambda (_, count): -count)   lambda x: x[1]
    return ['_'.join(x) for x in list(gram.keys())]

n = 4   # the value of 'n' in ngram
words = 'ಕೆಲವರು ಏನೇನೋ ಗಾದೆ ಮಾತು ಕೆಲವರು ಏನೇನೋ ದ್ರಾವಿಡ ಭಾಷೆಗಳಲ್ಲಿ ಪ್ರಾಮುಖ್ಯವುಳ್ಳ ಗಾದೆ ಮಾತು ಕೆಲವರು ಭಾಷೆಯೂ ಭಾರತದ'
word_trigram = generateNgram(words.split(), n)
print("Word trigrams : ", word_trigram, '\n')

# the syllables given below are of these words - ಏನೇನೋಕೆಲವರು
syllable = ['ಏ', 'ನೇ', 'ನೋ', 'ಕೆ', 'ಲ', 'ವ', 'ರು']
syl_trigram = generateNgram(syllable, n)
print("Syllable trigrams : ", syl_trigram, '\n')

# the character components of the word - ಏನೇನೋಕೆಲವರು
char_comp = ['ಏ', 'ನ', 'ೇ', 'ನ', 'ೋ', 'ಕ', 'ೆ', 'ಲ', 'ವ', 'ರ', 'ು']
cc_trigram = generateNgram(char_comp, n)
print("Char_Components trigrams : ", cc_trigram)


Word trigrams :  ['ಕೆಲವರು_ಏನೇನೋ_ಗಾದೆ_ಮಾತು', 'ಏನೇನೋ_ಗಾದೆ_ಮಾತು_ಕೆಲವರು', 'ಗಾದೆ_ಮಾತು_ಕೆಲವರು_ಏನೇನೋ', 'ಮಾತು_ಕೆಲವರು_ಏನೇನೋ_ದ್ರಾವಿಡ', 'ಕೆಲವರು_ಏನೇನೋ_ದ್ರಾವಿಡ_ಭಾಷೆಗಳಲ್ಲಿ', 'ಏನೇನೋ_ದ್ರಾವಿಡ_ಭಾಷೆಗಳಲ್ಲಿ_ಪ್ರಾಮುಖ್ಯವುಳ್ಳ', 'ದ್ರಾವಿಡ_ಭಾಷೆಗಳಲ್ಲಿ_ಪ್ರಾಮುಖ್ಯವುಳ್ಳ_ಗಾದೆ', 'ಭಾಷೆಗಳಲ್ಲಿ_ಪ್ರಾಮುಖ್ಯವುಳ್ಳ_ಗಾದೆ_ಮಾತು', 'ಪ್ರಾಮುಖ್ಯವುಳ್ಳ_ಗಾದೆ_ಮಾತು_ಕೆಲವರು', 'ಗಾದೆ_ಮಾತು_ಕೆಲವರು_ಭಾಷೆಯೂ', 'ಮಾತು_ಕೆಲವರು_ಭಾಷೆಯೂ_ಭಾರತದ'] 

Syllable trigrams :  ['ಏ_ನೇ_ನೋ_ಕೆ', 'ನೇ_ನೋ_ಕೆ_ಲ', 'ನೋ_ಕೆ_ಲ_ವ', 'ಕೆ_ಲ_ವ_ರು'] 

Char_Components trigrams :  ['ಏ_ನ_ೇ_ನ', 'ನ_ೇ_ನ_ೋ', 'ೇ_ನ_ೋ_ಕ', 'ನ_ೋ_ಕ_ೆ', 'ೋ_ಕ_ೆ_ಲ', 'ಕ_ೆ_ಲ_ವ', 'ೆ_ಲ_ವ_ರ', 'ಲ_ವ_ರ_ು']


In [36]:
from libindic.syllabifier import Syllabifier
instance = Syllabifier()



In [37]:
def feat(sent):
    words=sent.split(' ')
    try:
        syllables=instance.syllabify(sent)
    except:
        syllables=['NO']
    chars=[x for x in sent]
    ngrams=[]
    '''for i in range (1,2):
        ngrams.extend(generateNgram(words,i))'''
    for i in range(1,8):
        ngrams.extend(generateNgram(syllables,i))
    '''for i in range(1,5):
        ngrams.extend(generateNgram(chars,i))'''

    #print('words= ',words)
    #print('syllables= ', syllables)
    #print('chars= ',chars)
    return ngrams

In [38]:
feat('ಏನೇನೋಕೆಲವರು')

['ಏ',
 'ನೇ',
 'ನೋ',
 'ಕೆ',
 'ಲ',
 'ವ',
 'ರು',
 'ಏ_ನೇ',
 'ನೇ_ನೋ',
 'ನೋ_ಕೆ',
 'ಕೆ_ಲ',
 'ಲ_ವ',
 'ವ_ರು',
 'ಏ_ನೇ_ನೋ',
 'ನೇ_ನೋ_ಕೆ',
 'ನೋ_ಕೆ_ಲ',
 'ಕೆ_ಲ_ವ',
 'ಲ_ವ_ರು',
 'ಏ_ನೇ_ನೋ_ಕೆ',
 'ನೇ_ನೋ_ಕೆ_ಲ',
 'ನೋ_ಕೆ_ಲ_ವ',
 'ಕೆ_ಲ_ವ_ರು',
 'ಏ_ನೇ_ನೋ_ಕೆ_ಲ',
 'ನೇ_ನೋ_ಕೆ_ಲ_ವ',
 'ನೋ_ಕೆ_ಲ_ವ_ರು',
 'ಏ_ನೇ_ನೋ_ಕೆ_ಲ_ವ',
 'ನೇ_ನೋ_ಕೆ_ಲ_ವ_ರು',
 'ಏ_ನೇ_ನೋ_ಕೆ_ಲ_ವ_ರು']

In [39]:
vectorizer = TfidfVectorizer(
    sublinear_tf=True,
    strip_accents='unicode',
    min_df=1,
    analyzer=feat)
vectorizer.fit(list(train.cleaned))

train_features = vectorizer.transform(train.cleaned)
dev_features = vectorizer.transform(dev.cleaned)
test_features=vectorizer.transform(test.cleaned)

In [40]:
from sklearn.ensemble import VotingClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.svm import LinearSVC,SVC
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report, f1_score


lr=LogisticRegression(C=50)
lr.fit(train_features, y_train)
y_pred = lr.predict(dev_features) 
print(classification_report(y_dev, y_pred,digits=2))

                precision    recall  f1-score   support

Mixed_feelings       0.59      0.31      0.41       102
      Negative       0.70      0.59      0.64       237
      Positive       0.75      0.87      0.80       706
 not-malayalam       0.84      0.83      0.84       141
 unknown_state       0.75      0.72      0.73       580

      accuracy                           0.75      1766
     macro avg       0.73      0.66      0.69      1766
  weighted avg       0.74      0.75      0.74      1766



/home/vijay/Desktop/notebook/jupyterenv/lib/python3.8/site-packages/sklearn/linear_model/_logistic.py:763: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


In [41]:
y_pred = lr.predict(test_features)
test1=test
test1['label']=y_pred
test1=test1.drop(['cleaned'],axis=1)
test1.to_csv(lang+'_lr_syllabel.tsv',sep='\t')
print(len(test1))
test1.head()


1962


,id,text,label
0,Mal_1,Teaserinu kurach samayamkoodi mathram. Cant wa...,unknown_state
1,Mal_2,അപ്പോൾ കഥയുടെ റൂട്ട് മാറിയല്ലോ...,unknown_state
2,Mal_3,മൂത്തോൻ ട്രൈലെർ Trending List വരാത്തത് എന്താ ?,Positive
3,Mal_4,Nowadays 944k views is considered as 1M views ...,not-malayalam
4,Mal_5,Maass.trailer ennu paranja ithaanU makkale.......,Positive


In [42]:
svm_clf=SVC(kernel='linear',probability=True)
svm_clf.fit(train_features, y_train)
y_pred = svm_clf.predict(dev_features)
print(classification_report(y_dev, y_pred))

                precision    recall  f1-score   support

Mixed_feelings       0.67      0.22      0.33       102
      Negative       0.71      0.57      0.63       237
      Positive       0.73      0.88      0.80       706
 not-malayalam       0.87      0.80      0.83       141
 unknown_state       0.75      0.73      0.74       580

      accuracy                           0.75      1766
     macro avg       0.75      0.64      0.67      1766
  weighted avg       0.74      0.75      0.73      1766



In [43]:
y_pred = svm_clf.predict(test_features)
test1=test
test1['label']=y_pred
test1=test1.drop(['cleaned'],axis=1)
test1.to_csv(lang+'_lsvc_syllabel.tsv',sep='\t')
print(len(test1))
test1.head()


1962


,id,text,label
0,Mal_1,Teaserinu kurach samayamkoodi mathram. Cant wa...,unknown_state
1,Mal_2,അപ്പോൾ കഥയുടെ റൂട്ട് മാറിയല്ലോ...,unknown_state
2,Mal_3,മൂത്തോൻ ട്രൈലെർ Trending List വരാത്തത് എന്താ ?,Positive
3,Mal_4,Nowadays 944k views is considered as 1M views ...,not-malayalam
4,Mal_5,Maass.trailer ennu paranja ithaanU makkale.......,Positive


In [44]:
'''from sklearn.naive_bayes import GaussianNB, MultinomialNB

clf=MultinomialNB()
clf.fit(train_features, y_train)
y_pred = clf.predict(dev_features)
print(classification_report(y_dev, y_pred))'''

'from sklearn.naive_bayes import GaussianNB, MultinomialNB\n\nclf=MultinomialNB()\nclf.fit(train_features, y_train)\ny_pred = clf.predict(dev_features)\nprint(classification_report(y_dev, y_pred))'

In [45]:
from sklearn.neural_network import MLPClassifier
mlp=MLPClassifier(hidden_layer_sizes=(150,100,50), max_iter=300,activation = 'relu',solver='adam',random_state=1)
mlp.fit(train_features, y_train)
y_pred = mlp.predict(dev_features) 
print(classification_report(y_dev, y_pred,digits=2))

                precision    recall  f1-score   support

Mixed_feelings       0.49      0.38      0.43       102
      Negative       0.64      0.57      0.60       237
      Positive       0.75      0.82      0.78       706
 not-malayalam       0.78      0.76      0.77       141
 unknown_state       0.69      0.67      0.68       580

      accuracy                           0.71      1766
     macro avg       0.67      0.64      0.65      1766
  weighted avg       0.70      0.71      0.70      1766



In [46]:
y_pred = mlp.predict(test_features)
test1=test
test1['label']=y_pred
test1=test1.drop(['cleaned'],axis=1)
test1.to_csv(lang+'_mlp_syllabel.tsv',sep='\t')
print(len(test1))
test1.head()


1962


,id,text,label
0,Mal_1,Teaserinu kurach samayamkoodi mathram. Cant wa...,unknown_state
1,Mal_2,അപ്പോൾ കഥയുടെ റൂട്ട് മാറിയല്ലോ...,Positive
2,Mal_3,മൂത്തോൻ ട്രൈലെർ Trending List വരാത്തത് എന്താ ?,Positive
3,Mal_4,Nowadays 944k views is considered as 1M views ...,not-malayalam
4,Mal_5,Maass.trailer ennu paranja ithaanU makkale.......,Positive


In [47]:
from sklearn.ensemble import VotingClassifier
from sklearn.neural_network import MLPClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.svm import LinearSVC,SVC
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report, f1_score
from sklearn.multiclass import OneVsRestClassifier
import xgboost as xgb
from sklearn.linear_model import SGDClassifier



#mnb=MultinomialNB()
mlp=MLPClassifier(hidden_layer_sizes=(150,100,50), max_iter=300,activation = 'relu',solver='adam',random_state=1)
xgb_clf=SVC(kernel='linear',probability=True)
lr=LogisticRegression()
estimators=[('lr', lr), ('mlp', mlp),('xgb', xgb_clf)]
clf = VotingClassifier(estimators, voting='soft')
ensemble =clf


ensemble.fit(train_features, y_train)

y_pred = ensemble.predict(dev_features) 
print(classification_report(y_dev, y_pred,digits=4))



/home/vijay/Desktop/notebook/jupyterenv/lib/python3.8/site-packages/sklearn/linear_model/_logistic.py:763: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


                precision    recall  f1-score   support

Mixed_feelings     0.5588    0.3725    0.4471       102
      Negative     0.6881    0.5865    0.6333       237
      Positive     0.7516    0.8399    0.7933       706
 not-malayalam     0.8071    0.8014    0.8043       141
 unknown_state     0.7231    0.7069    0.7149       580

      accuracy                         0.7322      1766
     macro avg     0.7058    0.6615    0.6786      1766
  weighted avg     0.7270    0.7322    0.7270      1766



In [48]:
y_pred = ensemble.predict(test_features)
test1=test
test1['label']=y_pred
test1=test1.drop(['cleaned'],axis=1)
test1.to_csv(lang+'_ensemble_charSeq.tsv',sep='\t')
print(len(test1))
test1.head()


1962


,id,text,label
0,Mal_1,Teaserinu kurach samayamkoodi mathram. Cant wa...,unknown_state
1,Mal_2,അപ്പോൾ കഥയുടെ റൂട്ട് മാറിയല്ലോ...,Positive
2,Mal_3,മൂത്തോൻ ട്രൈലെർ Trending List വരാത്തത് എന്താ ?,Positive
3,Mal_4,Nowadays 944k views is considered as 1M views ...,not-malayalam
4,Mal_5,Maass.trailer ennu paranja ithaanU makkale.......,Positive


In [49]:
##char Sequences

In [50]:
from nltk import everygrams

def ngram_extractor(sent):
    return [''.join(ng) for ng in everygrams(sent.replace(' ', '_ _'), 1, 6) 
            if ' ' not in ng and '\n' not in ng and ng != ('_',)]



import os
import numpy
import pandas as pd
import re
import glob
from nltk.tokenize import word_tokenize
from nltk import FreqDist
from nltk import trigrams
from nltk import bigrams
from nltk import ngrams
from nltk.collections import defaultdict
#from nltk.classify.scikitlearn import SklearnClassifier
from sklearn.linear_model import LogisticRegression
#from gensim.models import KeyedVectors
from collections import Counter 
from nltk.classify.util import accuracy
from sklearn.metrics import precision_score, recall_score, f1_score, classification_report,accuracy_score
import pandas as pd
import nltk
#import gensim
#import collections

import unicodedata
#from string import punctuation
#from nltk.tokenize import WhitespaceTokenizer
from nltk import ngrams
from operator import itemgetter # The operator module exports a set of efficient functions corresponding to the intrinsic operators of Python. For example, operator.add(x, y) is equivalent to the expression x + y.


def get_word_ngrams(tokens, n):
    ngrams_list = []
    ngrams_list.append(list(ngrams(tokens, n)))
    ngrams_flat_tuples = [ngram for ngram_list in ngrams_list for ngram in ngram_list]
    format_string = '%s'
    for i in range(1, n):
        format_string += (' %s')
    ngrams_list_flat = [format_string % ngram_tuple for ngram_tuple in ngrams_flat_tuples]
    return ngrams_list_flat

def get_char_ngrams(word, n):
    ngrams_list = []
    word = re.sub(r'ς', 'σ', word)
    ngrams_list.append(list(ngrams(word, n, pad_left=True, pad_right=True, left_pad_symbol='_', right_pad_symbol='_')))
    
    # Removing redundant ngrams:
    if (n > 2):
        redundant_combinations = n - 2
        ngrams_list = [ngram_list[redundant_combinations : -redundant_combinations] for ngram_list in ngrams_list]
    
    ngrams_flat_tuples = [ngram for ngram_list in ngrams_list for ngram in ngram_list]
    format_string = ''
    for i in range(0, n):
        format_string += ('%s')
    ngrams_list_flat = [format_string % ngram_tuple for ngram_tuple in ngrams_flat_tuples]
    return ngrams_list_flat

# Feature extractor
def get_ngram_features(sent): # The reason I do not use NLTK's everygrams to extract the features quickly is because the behavior of my n-gram extractor is modified to remove redundant n-grams. Also, I need to label word and char n-grams to avoid ambiguity
  sent=sent.lower()
  sentence_tokens = sent.split(' ')


  freq_4grams = {}
  '''SNgram_obj=SNgramExtractor(sent,meta_tag='original',trigram_flag='yes',nlp_model=None)
  output=SNgram_obj.get_SNgram()'''

  # Word snbigrams
  ngrams = get_word_ngrams(sentence_tokens, 1)
  
  for ngram in ngrams:
    if ngram not in freq_4grams:
      freq_4grams.update({ngram:1})
  '''for word in sentence_tokens:
    ngrams=get_char_ngrams(word,2)
    for ngram in ngrams: 
        if ngram not in freq_4grams: 
            freq_4grams.update({ngram:1})'''
    
  '''# Word snTrigrams
  ngrams = get_word_ngrams(sentence_tokens, 3)
  
  for ngram in ngrams:
    if ngram not in freq_4grams:
      freq_4grams.update({ngram:1})'''

  ngrams = ngram_extractor(sent)
  ngrams=[remove_(t) for t in ngrams]

  #freq_4grams = {}

  for ngram in ngrams:
    if ngram not in freq_4grams:
        freq_4grams.update({ngram: 1})
    else:
        ngram_occurrences = freq_4grams[ngram]
        freq_4grams.update({ngram: ngram_occurrences + 1})
        

  freq_4grams_sorted = sorted(freq_4grams.items(), key=itemgetter(1), reverse=True)#[0:300] # We only keep the 300 most popular n-grams. This was suggested in the original paper written about n-grams.
  
   

    
  return list(dict(freq_4grams_sorted).keys())
def remove_(t):
  if t[0]=='_':t=t[1:]
  if t[-1]=='_':t=t[:-1]
  return t

train_set_sents = train.cleaned
y_train = train.category
dev_set_sents =dev.cleaned
y_dev = dev.category
test_set_sents=test.cleaned

In [54]:
from sklearn.feature_extraction.text import CountVectorizer,TfidfVectorizer

count_vect = TfidfVectorizer(analyzer=get_ngram_features)

train_features = count_vect.fit_transform(train_set_sents)
dev_features = count_vect.transform(dev_set_sents)
test_features = count_vect.transform(test_set_sents)


In [55]:
from sklearn.ensemble import VotingClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.svm import LinearSVC,SVC
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report, f1_score


lr=LogisticRegression(C=50)
lr.fit(train_features, y_train)
y_pred = lr.predict(dev_features) 
print(classification_report(y_dev, y_pred,digits=2))

                precision    recall  f1-score   support

Mixed_feelings       0.46      0.38      0.42       102
      Negative       0.70      0.57      0.63       237
      Positive       0.77      0.83      0.80       706
 not-malayalam       0.81      0.81      0.81       141
 unknown_state       0.73      0.73      0.73       580

      accuracy                           0.74      1766
     macro avg       0.69      0.67      0.68      1766
  weighted avg       0.73      0.74      0.73      1766



/home/vijay/Desktop/notebook/jupyterenv/lib/python3.8/site-packages/sklearn/linear_model/_logistic.py:763: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


In [56]:
y_pred = lr.predict(test_features)
test1=test
test1['label']=y_pred
test1=test1.drop(['cleaned'],axis=1)
test1.to_csv(lang+'_lr_charSeq.tsv',sep='\t')
print(len(test1))
test1.head()


1962


,id,text,label
0,Mal_1,Teaserinu kurach samayamkoodi mathram. Cant wa...,unknown_state
1,Mal_2,അപ്പോൾ കഥയുടെ റൂട്ട് മാറിയല്ലോ...,Positive
2,Mal_3,മൂത്തോൻ ട്രൈലെർ Trending List വരാത്തത് എന്താ ?,Positive
3,Mal_4,Nowadays 944k views is considered as 1M views ...,not-malayalam
4,Mal_5,Maass.trailer ennu paranja ithaanU makkale.......,Positive


In [57]:
svm_clf=SVC(kernel='linear',probability=True)
svm_clf.fit(train_features, y_train)
y_pred = svm_clf.predict(dev_features)
print(classification_report(y_dev, y_pred))

                precision    recall  f1-score   support

Mixed_feelings       0.60      0.31      0.41       102
      Negative       0.72      0.57      0.64       237
      Positive       0.75      0.86      0.80       706
 not-malayalam       0.85      0.83      0.84       141
 unknown_state       0.74      0.75      0.74       580

      accuracy                           0.75      1766
     macro avg       0.73      0.66      0.69      1766
  weighted avg       0.74      0.75      0.74      1766



In [58]:
y_pred = svm_clf.predict(test_features)
test1=test
test1['label']=y_pred
test1=test1.drop(['cleaned'],axis=1)
test1.to_csv(lang+'_lsvc_charSeq.tsv',sep='\t')
print(len(test1))
test1.head()


1962


,id,text,label
0,Mal_1,Teaserinu kurach samayamkoodi mathram. Cant wa...,unknown_state
1,Mal_2,അപ്പോൾ കഥയുടെ റൂട്ട് മാറിയല്ലോ...,unknown_state
2,Mal_3,മൂത്തോൻ ട്രൈലെർ Trending List വരാത്തത് എന്താ ?,Positive
3,Mal_4,Nowadays 944k views is considered as 1M views ...,not-malayalam
4,Mal_5,Maass.trailer ennu paranja ithaanU makkale.......,Positive


In [59]:
from sklearn.neural_network import MLPClassifier
mlp=MLPClassifier(hidden_layer_sizes=(150,100,50), max_iter=300,activation = 'relu',solver='adam',random_state=1)
mlp.fit(train_features, y_train)
y_pred = mlp.predict(dev_features) 
print(classification_report(y_dev, y_pred,digits=2))

                precision    recall  f1-score   support

Mixed_feelings       0.41      0.35      0.38       102
      Negative       0.60      0.56      0.58       237
      Positive       0.74      0.80      0.77       706
 not-malayalam       0.72      0.78      0.75       141
 unknown_state       0.70      0.65      0.68       580

      accuracy                           0.69      1766
     macro avg       0.63      0.63      0.63      1766
  weighted avg       0.69      0.69      0.69      1766



In [60]:
y_pred = mlp.predict(test_features)
test1=test
test1['label']=y_pred
test1=test1.drop(['cleaned'],axis=1)
test1.to_csv(lang+'_mlp_charSeq.tsv',sep='\t')
print(len(test1))
test1.head()


1962


,id,text,label
0,Mal_1,Teaserinu kurach samayamkoodi mathram. Cant wa...,not-malayalam
1,Mal_2,അപ്പോൾ കഥയുടെ റൂട്ട് മാറിയല്ലോ...,Positive
2,Mal_3,മൂത്തോൻ ട്രൈലെർ Trending List വരാത്തത് എന്താ ?,Negative
3,Mal_4,Nowadays 944k views is considered as 1M views ...,not-malayalam
4,Mal_5,Maass.trailer ennu paranja ithaanU makkale.......,Positive


In [61]:
from sklearn.ensemble import VotingClassifier
from sklearn.neural_network import MLPClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.svm import LinearSVC,SVC
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report, f1_score
from sklearn.multiclass import OneVsRestClassifier
import xgboost as xgb
from sklearn.linear_model import SGDClassifier



#mnb=MultinomialNB()
mlp=MLPClassifier(hidden_layer_sizes=(150,100,50), max_iter=300,activation = 'relu',solver='adam',random_state=1)
xgb_clf=SVC(kernel='linear',probability=True)
lr=LogisticRegression()
estimators=[('lr', lr), ('mlp', mlp),('xgb', xgb_clf)]
clf = VotingClassifier(estimators, voting='soft')
ensemble =clf


ensemble.fit(train_features, y_train)

y_pred = ensemble.predict(dev_features) 
print(classification_report(y_dev, y_pred,digits=4))

/home/vijay/Desktop/notebook/jupyterenv/lib/python3.8/site-packages/sklearn/linear_model/_logistic.py:763: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


                precision    recall  f1-score   support

Mixed_feelings     0.4684    0.3627    0.4088       102
      Negative     0.6749    0.5781    0.6227       237
      Positive     0.7554    0.8357    0.7935       706
 not-malayalam     0.7842    0.7730    0.7786       141
 unknown_state     0.7287    0.7086    0.7185       580

      accuracy                         0.7271      1766
     macro avg     0.6823    0.6516    0.6644      1766
  weighted avg     0.7216    0.7271    0.7226      1766



In [62]:
y_pred = ensemble.predict(test_features)
test1=test
test1['label']=y_pred
test1=test1.drop(['cleaned'],axis=1)
test1.to_csv(lang+'_ensemble_charSeq.tsv',sep='\t')
print(len(test1))
test1.head()


1962


,id,text,label
0,Mal_1,Teaserinu kurach samayamkoodi mathram. Cant wa...,unknown_state
1,Mal_2,അപ്പോൾ കഥയുടെ റൂട്ട് മാറിയല്ലോ...,Positive
2,Mal_3,മൂത്തോൻ ട്രൈലെർ Trending List വരാത്തത് എന്താ ?,Positive
3,Mal_4,Nowadays 944k views is considered as 1M views ...,not-malayalam
4,Mal_5,Maass.trailer ennu paranja ithaanU makkale.......,Positive
